# 本筆記將帶大家用Keras建模
# 此範例使用MNIST手寫數字資料集

[I. 資料準備](#I.-資料準備)

[II. Softmax Regression](#II.-Softmax-Regression)

[III. AlexNet-like Network](#III.-AlexNet-like-Network)

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import pandas as pd
import sklearn
import os

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='mxnet')
warnings.filterwarnings("ignore", category=DeprecationWarning, module='keras.utils')

### 下載資料

In [ ]:
from zipfile import ZipFile
from urllib.request import urlretrieve

print('retrieving MNIST data...')
theurl = 'https://github.com/chi-hung/PythonTutorial/raw/master/datasets/mnist.tar.gz'
filename='mnist.tar.gz'
name, hdrs = urlretrieve(theurl, filename)

print('extracting MNIST data...')
res=!tar -xvf {filename}

print('done!')

---

### I. 資料準備

#### 載入圖片

In [ ]:
def filePathsGen(rootPath):
    paths=[]
    dirs=[]
    for dirPath,dirNames,fileNames in os.walk(rootPath):
        for fileName in fileNames:
            fullPath=os.path.join(dirPath,fileName)
            paths.append((int(dirPath[len(rootPath) ]),fullPath))
        dirs.append(dirNames)
    return dirs,paths

In [ ]:
dirs,paths=filePathsGen('mnist/') #載入圖片路徑

In [ ]:
dfPath=pd.DataFrame(paths,columns=['class','path']) #圖片路徑存成Pandas資料表
dfPath.head(5) # 看資料表前5個row

#### 利用Pandas, 可迅速了解每個資料夾裡面有幾張圖片

In [ ]:
#依照class分群後，數各群的數量，並繪圖
dfCountPerClass=dfPath.groupby('class').count()
dfCountPerClass.rename(columns={'path':'amount of figures'},inplace=True)
dfCountPerClass.plot(kind='bar',rot=0)

####  將圖片路徑資訊分成train($70\%$), val($10\%$), test($10\%$)

In [ ]:
dfFiguresShuffled=dfPath.sample(frac=1) # 打亂一下path data

#dfFrac=dfFiguresShuffled
dfFrac=dfFiguresShuffled.sample(frac=0.3) # 以下範例，我們只取原資料集的30%來做使用，
                                            # 這是為了利於在課堂中快速演練。
    
train=dfFrac.sample(frac=0.8) # 將path data隨機取樣，80%的path data當train
test=dfFrac.drop(train.index) # 20%的path data當test

trainVal=train.sample(frac=1/8) # 將train再切1/8做驗證用資料, 存至trainVal
train=train.drop(trainVal.index)# 將train的7/8留著，丟去剛切出去的1/8

#最終，整體資料拿70%當train, 10%當train_val, 20%當test。
print('shape(all figures)=\t\t',dfPath.shape)
print('shape(fraction of figures)=\t',dfFrac.shape)
print('shape(train)=\t\t\t',train.shape)
print('shape(trainVal)=\t\t',trainVal.shape)
print('shape(test)=\t\t\t',test.shape)

#隨便抓三張圖來看
for j in range(3):
    img=plt.imread(train['path'].iloc[j])
    plt.imshow(img)
    plt.show()

####  將圖片載入，存成數值矩陣

In [ ]:
def dataLoad(dfPath):
    paths=dfPath['path'].values
    x=np.zeros((len(paths),28,28),dtype=np.float32 )

    for j in range(len(paths)):
        x[j,:,:]=plt.imread(paths[j])/255

    y=dfPath['class'].values
    return x,y

In [ ]:
trainX,trainY=dataLoad(train)
trainValX,trainValY=dataLoad(trainVal)
testX,testY=dataLoad(test)

In [ ]:
print('train:\t',trainX.shape,trainY.shape)
print('trainVal:',trainValX.shape,trainValY.shape)
print('test:\t',testX.shape,testY.shape)

---

## II. Softmax Regression

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Flatten
from keras.optimizers import SGD

將y 轉成one hot形式

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
trainYOneHot=np.float32( enc.fit_transform(trainY.reshape(-1,1)) \
                .toarray() )

trainValYOneHot=np.float32( enc.fit_transform(trainValY.reshape(-1,1)) \
                   .toarray() )

testYOneHot=np.float32( enc.fit_transform(testY.reshape(-1,1)) \
               .toarray() )

In [ ]:
print('train:\t',trainX.shape,trainY.shape)
print('trainVal:',trainValX.shape,trainValY.shape)
print('test:\t',testX.shape,testY.shape)

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=(28,28)))
model.add(Dense(10, activation='softmax') )

sgd=SGD(lr=0.2, momentum=0.0, decay=0.0)
model.compile(optimizer='sgd',
      loss='categorical_crossentropy',
      metrics=['accuracy'])

In [ ]:
hist=model.fit(trainX, trainYOneHot,
               epochs=20, batch_size=128,
               validation_data=(trainValX,trainValYOneHot))

In [ ]:
score = model.evaluate(testX, testYOneHot, batch_size=128)

In [ ]:
score # [loss , accuracy]

In [ ]:
for j in range(5):
    print( model.predict(trainX[j:j+1,:]).argmax() )
    print( trainYOneHot[j].argmax() )
    print()

In [ ]:
plt.plot(hist.history['acc'],ms=5,marker='o',label='accuracy')
plt.plot(hist.history['val_acc'],ms=5,marker='o',label='val accuracy')
plt.legend()
plt.show()

In [ ]:
model.summary()

儲存模型和權重

In [ ]:
import json
with open('first_try.json', 'w') as jsOut:
    json.dump(model.to_json(), jsOut)

model.save_weights('first_try.h5')

載入存好的模型和權重

In [ ]:
from keras.models import model_from_json

In [ ]:
with open('first_try.json', 'r') as jsIn:
    modelJson=json.load(jsIn)
    
modelLoaded=model_from_json(modelJson)
modelLoaded.load_weights('first_try.h5')

In [ ]:
modelLoaded.summary()

In [ ]:
predY=model.predict(testX).argmax(axis=1)
from sklearn.metrics import classification_report
print( classification_report(predY,testY) )

In [ ]:
trainX.shape

In [ ]:
trainValX.shape

In [ ]:
testX.shape

## III. AlexNet-like Network

In [ ]:
trainX=trainX.reshape(trainX.shape[0],1,trainX.shape[1],trainX.shape[2])
trainValX=trainValX.reshape(trainValX.shape[0],1,trainValX.shape[1],trainValX.shape[2])
testX=testX.reshape(testX.shape[0],1,testX.shape[1],testX.shape[2])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Conv2D, MaxPooling2D
from keras.layers import Activation
from keras.optimizers import SGD,Adam,Adamax
#from keras.losses import categorical_crossentropy

input_shape=(1,28,28)

model = Sequential()
#conv1
model.add(Conv2D(32, (3,3),
                 activation='relu',
                 input_shape=(1,28,28)))
#conv2
model.add(Conv2D(64, (3,3), activation='relu')
         )
#pool1
model.add(MaxPooling2D(pool_size=(2, 2))
         )
#dropout1
model.add(Dropout(0.5))
model.add(Flatten())
#dense1
model.add(Dense(128, activation='relu')
         )
#dropout2
model.add(Dropout(0.5))
#dense2
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.01,momentum=0.1),
              metrics=['accuracy'],
              context=['gpu(0)'])

In [ ]:
%%time 
hist=model.fit(trainX, trainYOneHot, 
               epochs=20,
               batch_size=32,
               validation_data=(trainValX,trainValYOneHot),
              )

In [ ]:
plt.plot(hist.history['acc'],ms=5,marker='o',label='accuracy')
plt.plot(hist.history['val_acc'],ms=5,marker='o',label='val accuracy')
plt.legend()
plt.show()

In [ ]:
predY=model.predict(testX).argmax(axis=1)
from sklearn.metrics import classification_report
print( classification_report(predY,testY) )